In [43]:
import pandas as pd
df = pd.read_excel('data/train.xlsx', sheet_name='train')
# df_tag = df['tags']
# df_tag = df_tag.head()

In [44]:
import re
from collections import Counter

# 'tag' 컬럼 전처리: 정규표현식을 통해 문자만 남기기
def clean_tag(tag):
    return re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '', tag)  # 문자만 남기기

In [45]:
# 모든 태그를 수집할 리스트
all_tags = []

# 각 행의 태그를 처리
processed_tags = []
for tags in df['tags']:
    if isinstance(tags, str):  # 문자열인지 확인
        # 문자열을 쉼표로 분리하여 개별 태그 추출
        tags_list = tags.split(',')
        # 각 태그를 전처리
        cleaned_tags = [clean_tag(tag) for tag in tags_list]
        # 전처리된 태그를 다시 문자열로 결합하여 리스트에 추가
        processed_tags.append(','.join(cleaned_tags))
        # 모든 태그를 수집
        all_tags.extend(cleaned_tags)
    else:
        processed_tags.append('')

# 전처리된 태그를 기존 'tag' 컬럼에 반영
df['tags'] = processed_tags


In [ ]:
# 각 태그의 빈도수 계산
tag_counter = Counter(all_tags)

# 빈도수 출력
for tag, count in tag_counter.items():
    print(f'Tag: {tag}, Count: {count}')

# 결과를 데이터프레임으로 변환하여 보기
tag_counts_df = pd.DataFrame(tag_counter.items(), columns=['Tag', 'Count'])
tag_counts_df.to_excel('data/카운터.xlsx')

In [47]:
# 불용어 리스트
stopwords = [
'곡', '노래', '아이유', '트와이스', '엑소', '레드벨벳', '황덕호', '한동윤', '정미애', '미스트롯', '워너원', '기리보이', '마마무', 
'비투비', '소녀시대', '유산슬', '한지훈', '볼빨간사춘기', '내한공연', '종현', '박효신', '샤이니', '하트시그널', '태연', '크러쉬', 
'효리네민박', '방탄', '뉴이스트', '응답하라', '할', '리스트', '듣는', '모음', '나', '기전', '성', '차트', '들', '가', '에서', 
'뮤직', '밴드', '숨은', '빌보드', '만', '을', '내', '장르', '송', '있는', '최신', '공연', '지', '살', '고막', '셋', '별', '기', 
'방탄소년단', '신곡', '듀엣', '너', '멜로디', '로', '알고싶은', '시', '빌', '목소리', '원', '를', '아티스트', '귀', '만의', 
'는', '내한', '히든트랙', '불문', '히트', '콘서트', '에센셜', '은', '멜론', '올', '후', '애창', '하', '핫', '싱어송라이터', 
'로운', '과', '해', '력', '가창력', '보이', '송가', '스', '바', '레전드', '프로', '오늘', '잘', '선곡', '사운드', '디', 
'귀르가즘', '없는', '다', '풀', '라', '슈즈', '딥', '고', '오', '하는', '와', '아', '스밍', '년', '오마이걸', '데이식스', 
'쫑', '아리아나그란데', '러블리쏘', '이효리', '러블리즈', '홍자', '지코', '빅뱅', '대표곡', '콜드플레이', '유재석', 
'델리민주', '고등래퍼', '블랙핑크', '빌보드핫차트', '년생', '웹진이즘', '비긴어게인', '나만알고싶은노래', '세트리스트', 
'아이돌탐구생활', '홍진영', '지역별', '내마음의사진', '빈지노', '내가좋아하는음악트랙', '헤이즈', '세븐틴', '윤종신', 
'띵곡모음', '쇼팽', '명곡모음', '이유진', '이상순', '에이핑크', '하이라이트', '보아', '년도노래', '프로듀스시즌', '역주행했으면하는노래', '아이들', '무한반복', '커버곡',
'아이돌수록곡', '아이돌숨은명곡', '엠씨더맥스', '장윤정', '토토가', '창모', '에드시런', '비틀즈', '잔나비', '호텔델루나', '임영웅',
'임창정', '박재범', '에프엑스', '멜로망스', '멜론차트', '이소라', '무한도전', '박효재', '랏밴뮤', '랏도의밴드뮤직',
'붸붸', '방탄소년단명곡', '인피니트', '켄드릭라마', '우주소녀', '백예린', '아이즈원', '폴킴', '영탁', '혁오', '자이언티',
'위너', '성시경', '동방신기', '올림픽', '젝스키스', '풀앨범', '윤하', '유희열', '슈퍼주니어', '케이윌', '년대가요', '듣도보도못한굿뮤직',
'공유', '그레이', '쇼미', '서재페', '에미넴', '김동률', '청하', '나만아는', '윤식당', '수록곡추천', '이루마', '아무로나미에',
'강다니엘', '핑클', '푸른밤종현입니다', '이달의소녀', '하이어뮤직', '씨스타', '알라딘', '년도생', '서울재즈페스티벌', '믹스나인',
'년도', '비스트', '가야씨', '추천해주고싶은노래', '더콰이엇', '프듀', '어디서들어본노래', '싸이', '원더걸스', '넉살',
'팝스타시즌', '방금그곡', '다비치', '웨하스구름', '조성진', '정다경', '김소유', '비와이', '그루비룸', '블락비', '검정치마',
'찰리푸스', '딩고뮤직', '신해철', '아이콘', '라디오헤드', '놀면뭐하니', '사랑의재개발', '정승환', '장범준', '버스커버스커', 
'정세운', '핫트랙', '필레이리스트', '원필', '우원재', '팬시차일드', '주년', '칠', '서태지', '굿피플', '한번쯤들어봤던노래',
'라흐마니노프', '투모로우바이투게더', '이태원클라쓰', '빅스', '드레이크', '체인스모커스', '나얼', '갓세븐', '김광석',
'머라이어캐리', '시대별', '월디페', '띵곡', '마이클잭슨', '저스디스', '소란', '식케이', '신예', '삼시세끼', '제목',
'슈퍼밴드', '뮤직', '비욘세', '제이지', '뷔', '정준일', '키드밀리', '에이프릴', '리메이크곡', '묘니', '비틀스', '솔로곡', 
'조명섭', '카더가든', '브라운아이즈', '브라운아이즈소울', '트로이시반', '아델', '박진영', '스윙스', '믿듣', '아비치',
'치즈', '브로콜리너마저', '프레디머큐리', '김이나', '워너비', '인기차트', '해쉬스완', '더뮤지션', '신보', '하유비', '에일리',
'매드무비', '윤상', '몬스타엑스', '지누락엔터테인먼트', '노라존스', '백지영', '규현', '윤도현', '슈베르트', '케이티페리',
'민냥', '두아리파', '지민', '옥상달빛', '이수', '샘스미스', '휘트니휴스턴', '레이디가가', '휘성', '박정현', '딩고', '태양의후예',
'페퍼톤스', '콘서트', '하성운', '우효', '구구단', '디스윅스', '양준일', '다이나믹듀오', '테일러스위프트', '꽃보다청춘', '에픽하이',
'엔시티', '엔시티드림', '스탠딩에그', '엠비션뮤직', '선미', '엑스원', '박보검', '한요한', '원스', '지드래곤', '이일우'
]

In [ ]:
# 불용어 정리 함수
def remove_stopwords(tag):
    words = tag.split(',')
    return ','.join([word for word in words if word not in stopwords])

# 'tag' 컬럼의 불용어 정리
df['tags'] = df['tags'].apply(remove_stopwords)

df['tags'].head(20)

In [ ]:
from konlpy.tag import Okt

# Okt 객체 생성
tokenizer = Okt()

# 명사, 형용사, 부사만 남기고 불용어 제거
def tokenize_and_filter(tags):
    tokenized_sentence = tokenizer.pos(tags)  # 토큰화 및 품사 태깅
    filtered_sentence = [word for word, pos in tokenized_sentence if pos in ['Noun', 'Adjective', 'Adverb'] and word not in stopwords]  # 명사, 형용사, 부사만 남기고 불용어 제거
    return filtered_sentence

# 토큰화 및 불용어 제거 적용
df['tags'] = df['tags'].apply(tokenize_and_filter)

# 결과 출력
df['tags'].head(20)

In [87]:
def tagCount(df, version):
    # 모든 태그를 하나의 리스트로 합침
    all_tags = [tag for sublist in df['tags'] for tag in sublist]  

    # 빈도수 계산
    tag_counts = Counter(all_tags)  

    # 결과 출력
    print(tag_counts)

    # Counter 객체를 DataFrame으로 변환하여 엑셀 파일에 저장
    tag_counts_df = pd.DataFrame(tag_counts.items(), columns=['Tag', 'Count'])
    tag_counts_df.to_excel(f'data/카운터{version}.xlsx', index=False)

In [ ]:
# 태그의 빈도수가 2 이상인 것만 필터링
filtered_tags = [tag for tag, count in tag_counts.items() if count > 21]

# 필터링된 태그를 가진 행만 선택하여 새로운 데이터프레임 생성
filtered_df = df[df['tags'].apply(lambda x: any(tag in filtered_tags for tag in x))]

# 필터링된 결과를 원래 데이터프레임에 할당하여 업데이트
df['tags'] = filtered_df['tags']

df['tags']

In [ ]:
# 결측치를 빈 리스트로 대체하여 처리
df['tags'] = df['tags'].fillna('').apply(lambda x: list(set(x)) if isinstance(x, list) else [])
# 'tags' 컬럼이 빈 리스트(결측치)인 행을 모두 제거
df = df[df['tags'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

df.info()

In [69]:
df_1st_processed = df
df.to_csv('data/train_전처리1.csv')

--- 2차 전처리 시작 ---
1) 불용어 다시 설정, 제거
2) 특정 단어 제외 한글자 단어 제거

In [77]:
stopwords = [
    '음악', '날', '때', '명곡', '추천', '년대', '월', '인기', '쇼미더머니', '명반', '있을',
    '전곡', '함', '걸', '더', '오피', '셜', '쉬', '인', '피', '렉스', '듀스', '빗',
    '없이', '슈가', '딘', '스쿨오브록', '움', '앤', '틴', '미', '골', '유', '어', '뭐', '테디',
    '그래미', '도깨비', '안', '바흐', '로움', '포', '인기가요', '션', '넬', '생', '워', 
    '묘', '배철수', '태양', '현빈', '노엘', '카라', '이빛나', '라이온킹', 
    '콜드', '도끼', '윤아', '지수', '다니엘', '갓띵곡', '이민혁', '추', '카이',
    '백현', '조이', '선우정아', '차이코프스키', '사무엘', '승연', '데이빗보위', '있어',
    '새소년', '지산', '마이클', '밥딜런', '메킷레', '민경훈', '토이', '소유', '나영석',
    '김호중', '히든싱어', '이지은', '정동원', '스텔라', '제이레빗', '김건모',
    '경리', '이문세', '최', '케이시', '드렁큰타이거', '벨벳', '유주', '자우림', '한스',
    '양현석', '현아', '안예은','그래미어워드', '릴러말즈', '플라이투더스카이', '나인뮤지스',
    '양요섭', '화요비', '이선희', '한해', '윤딴딴', '셀린디온', '신승훈', '알렌', '릭스',
    '테이', '효리','드림캐쳐', '권진아', '애즈원', '설리', '케이팝스타', '티파니', '백아연', '디오',
    '나훈아', '수란', '김연자', '크리스토퍼', '워너블', '씨잼', '안지영', '정유년', '이승환', '장민호',
    '케빈', '이찬원', '박서진', '유스케', '지윤', '로이킴', '에릭', '알렉', '미녀와야수', '라푼젤',
    '트래비스', '박지윤', '앰비션', '마크', '먼데이키즈', '린킨', '렌', '민현', '찬열', '스웨덴세탁소',
    '터보', '지아', '이하이', '에디', '정일훈', '이창섭', '서은광', '육성재', '마룬파이브',
    '머', '저', '뮤', '안드', '원디렉션', '마왕', '페노메코', '말리', '엘', '친',
    '서태지와아이들', '서현', '스콧', '신중현', '스티비원더', '엄선', '김광민',
    '잭슨', '엘튼존', '성유리', '문희준', '데이비드보위', '이진', '인터스텔라',
    '지펑크', '언니네이발관', '못', '덜', '쩌', '브라이언', '지은', '김경호', '시아',
    '차이콥스키', '라붐', '레디', '레오', '번스타인', '쌈디', '쿤스트', '딥플로우',
    '태티서', '스미스', '장미여관', '염따', '박경', '마룬', '강하늘', '백건우', '김효',
    '아린', '봉준호', '황민', '손열음', '나서연', '즘', '윤건', '실', '아스트로', '김태형',
    '무무', '샘김', '유키구라모토', '조용필', '김종현', '조성모', '박지민', '효', '반', '서지', 
    '프니엘', '캘빈', '예리', '레벨', '웬디', '슬기', '데이브레이크', '유투', '하현우',
    '심규선', '도운', '조수미', '넥스트', '박명수', '박지훈', '이승철', '존레논', '스테이션', '에드',
    '죠지', '켄지', '스크릴렉스', '스눕독', '한동근', '엔플라잉', '아이즈', '브라운', '이재원',
    '핱시', '마션', '한쇼', '위대', '롤링스톤', '오혁', '정연', '태후', '타임리스',
    '장국영', '김민종', '노르웨이숲', '가을방학', '용준형', '이기광', '윤두준',
    '장기하', '블랙아이드피스', '카밀라', '카베', '김원', '데이빗', '안재홍', '박우진',
    '프라이머리', '유재하', '오드리', '미카', '저스틴팀버레이크', '매드체스터',
    '엄지', '은하', '김필', '손예진', '이기용', '매드클라운', '파가니니', '오바마', '정경화',
    '박완규', '전람회', '김재환', '정규', '예지', '치타', '커트코베인', '너바나', '카디',
    '윤지성', '뮤직뱅크', '화사', '라이스', '브루노', '질라', '등등', 
    '수지', '이정아', '채영', '김석진', '때문', '씽', '라비', '현인', '레베럽', '아이린',
    '드림콘서트', '조정석', '가온차트', '슴스테', '던밀스', '김연아', '김나영', '보물상자',
    '에픽', '씨', '이수영', '권정열', '황성제', '우린', '재현', '알리샤', '산하엽', '가수만',
    '타이가', '캐', '탁', '겸', '조', '뱅', '폐막식', '악동뮤지션', '아미'
    ]

In [94]:
def remove_stopwords2(tag):
    if isinstance(tag, str):  # tag가 문자열인 경우
        words = tag.split(',')
    elif isinstance(tag, list):  # tag가 리스트인 경우
        words = tag
    else:
        return tag  # tag가 문자열도 리스트도 아닌 경우, 그대로 반환

    # 불용어를 제거한 리스트를 반환
    return [word for word in words if word not in stopwords]

def remove_short_words(tags):
    if isinstance(tags, list):
        return [word for word in tags if len(word) > 1 or word in one_word]
    return tags

def replace_typos(tags):
    if isinstance(tags, list):
        return [corrections.get(word, word) for word in tags]
    return tags

# 중복 단어 제거 함수
def remove_duplicates(tags):
    if isinstance(tags, list):
        return list(set(tags))
    return tags

In [ ]:
# 'tags' 컬럼의 불용어 정리
df['tags'] = df['tags'].apply(remove_stopwords2)

df['tags'].head(20)

In [85]:
one_word = ['꽃','꿈','끝','달','락','랩','록','멍','밤','밥','봄','비','삶','숨','숍','썸','술','춤','차','책','잠','집','팝','펍','흥','힙','힘','꿀']

In [ ]:
# 'tags' 컬럼에서 한 글자인 단어 제거 ('봄', '밤' 제외)
df['tags'] = df['tags'].apply(remove_short_words)

df['tags'].head(20)

In [96]:
tagCount(df, 4)

Counter({'기분전환': 16561, '감성': 14416, '휴식': 11361, '발라드': 11234, '잔잔한': 10506, '카페': 10271, '드라이브': 10222, '힐링': 10150, '새벽': 9269, '사랑': 9067, '밤': 9063, '힙합': 7489, '팝': 7132, '추억': 6838, '매장': 5861, '이별': 5555, '신나는': 5291, '설렘': 4949, '락': 4877, '여행': 4774, '회상': 4732, '가을': 4558, '겨울': 4516, '스트레스': 4185, '봄': 3694, '비': 3690, '랩': 3626, '재즈': 3623, '댄스': 3603, '여름': 3549, '슬픔': 3546, '뉴에이지': 3525, '알앤비': 3366, '산책': 2929, '분위기': 2834, '운동': 2793, '클래식': 2483, '일렉': 2362, '좋은': 2311, '피아노': 2224, '연주': 2107, '소울': 2105, '위로': 2036, '팝송': 1922, '클럽': 1836, '취향': 1766, '트렌디': 1629, '아이돌': 1583, '주말': 1539, '저격': 1534, '달달': 1450, '커피': 1422, '오후': 1387, '저녁': 1342, '그루브': 1339, '어쿠스틱': 1305, '혼자': 1282, '아침': 1264, '편안한': 1145, '우울': 1139, '국내': 1080, '감각': 1048, '몽환': 1015, '퇴근길': 1002, '집중': 1001, '일렉트로니카': 998, '가요': 987, '크리스마스': 982, '여유': 901, '음색': 895, '자장가': 893, '듣기': 883, '공부': 866, '기분': 861, '센치': 842, '그리움': 821, '캐롤': 798, '보컬': 771, '해외': 754, '출근길': 729, '따뜻한': 729, 

In [92]:
corrections = {
    '휠링': '힐링', '까페': '카페', '록': '락', '힙합엘이': '힙합', '갬성':'감성', '런닝':'러닝', '비올':'비', '쓸쓸하고':'쓸쓸', '흥겨':'흥', '쓸쓸하게':'쓸쓸',
    '째즈':'재즈', '리드미컬한':'리드미컬', '러닝머신':'러닝', '뮤직페스티벌':'페스티벌', '락페스티벌':'락', '런닝머신':'러닝', '봄봄봄':'봄', '합합':'힙합'
}

stopwords = [
    '가수', '일리', '스타', '성대', '역주행', '다중', '음격', '클리', '요즘', '앨범', '우리', '작곡', 
    '신인', '시즌', '웨이', '주의', '아웃', '순이', '랜드', '그때', '웹진', '커버', '두운', '기간', 
    '레코드', '타이틀곡', '숨은띵곡', '부록', '별책', '한번', '업데이트', '포스트', '겨울왕국', '정리', 
    '베토벤', '지미', '금지곡', '시대', '백송', '듀오', '유발', '오디오', '실력', '아이오', '질리', 
    '데이', '태그', '청곡', '월일', '월간', '화이트', '블루', '이나', '주간', '블랙', '많이', '데스', 
    '휘트니', '반복', '샘플링', '엔딩', '배우', '작사가', '주관', '명연', '계속', '리프', '컴백', 
    '있고싶을', '라인', '상반기', '뉴트', '편곡', '스웨', '거나', '어스', '애시드', '디락', '합정역', 
    '엔터테인먼트', '빅룸', '업뎃', '라이', '싱어', '어도', '레이', '라라', '로드', '에이', '사전', 
    '빅밴드', '맞이', '하드', '음악앨범', '플리', '불시', '팬송', '모차르트', '없음', '구석', '버즈', 
    '디스', '케이'
]

In [ ]:
df['tags'] = df['tags'].apply(replace_typos)
# df['tags'] = df['tags'].apply(remove_stopwords2)
df['tags'] = df['tags'].apply(remove_duplicates)

df['tags'].head(20)

In [107]:
def filter_tags(df, min_count):
    # 모든 태그를 하나의 리스트로 합침
    all_tags = [tag for sublist in df['tags'] for tag in sublist]
    
    # 빈도수 계산
    tag_counts = Counter(all_tags)
    
    # 빈도가 min_count보다 큰 태그들만 필터링
    filtered_tags = {tag for tag, count in tag_counts.items() if count > min_count}
    
    # 원래 데이터프레임의 tags 컬럼을 필터링된 태그들로 업데이트
    df['tags'] = df['tags'].apply(lambda tags: [tag for tag in tags if tag in filtered_tags])


In [120]:
# filter_tags(df, 21)
tagCount(df, 6)

Counter({'기분전환': 16561, '감성': 14416, '휴식': 11361, '발라드': 11234, '잔잔한': 10813, '카페': 10271, '드라이브': 10252, '힐링': 10150, '새벽': 9269, '사랑': 9195, '밤': 9083, '힙합': 7891, '팝': 7160, '추억': 6838, '매장': 5861, '신나는': 5638, '이별': 5555, '설렘': 5182, '락': 4884, '여행': 4774, '회상': 4732, '가을': 4558, '겨울': 4516, '스트레스': 4185, '댄스': 3949, '슬픔': 3948, '봄': 3947, '랩': 3712, '비': 3690, '재즈': 3631, '일렉': 3588, '여름': 3579, '뉴에이지': 3525, '알앤비': 3497, '산책': 2929, '분위기': 2834, '운동': 2793, '클래식': 2483, '피아노': 2224, '연주': 2128, '소울': 2105, '위로': 2036, '팝송': 1922, '클럽': 1836, '트렌디': 1714, '커피': 1625, '아이돌': 1583, '주말': 1539, '그루브': 1486, '우울': 1473, '달달': 1450, '아침': 1403, '오후': 1387, '저녁': 1342, '퇴근': 1325, '어쿠스틱': 1305, '혼자': 1282, '편안한': 1248, '센치': 1100, '국내': 1080, '감각': 1048, '몽환': 1015, '집중': 1001, '크리스마스': 992, '가요': 987, '따뜻함': 951, '여유': 901, '출근': 900, '음색': 895, '자장가': 893, '공부': 866, '캐롤': 863, '기분': 861, '그리움': 840, '쓸쓸함': 808, '영화': 797, '해외': 754, '흥': 711, '가사': 701, '일상': 684, '연인': 664, '시원': 65

In [ ]:
df['tags'] = df['tags'].fillna('').apply(lambda x: list(set(x)) if isinstance(x, list) else [])
df = df[df['tags'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

df.info()

In [123]:
df_3rd_processed = df
df_3rd_processed.to_csv('data/train_전처리3.csv')

-- 3차 전처리 --
1) 오타, 비슷한 단어 정리하기
2) 불용어 추가 정리

In [117]:
stopwords = [
    '복면', '취향', '저격', '다시', '듣기', '베스트', '개인', '깡패', '같은', '같이', '개취', '결산',
    '골든', '구분그룹', '내적', '다운', '대세', '대중가요', '대중음악', '데뷔', '도입', '듀엣곡', '디엠',
    '띵곡들', '라이브', '랜덤', '레어', '레이블', '로우', '루프', '리믹스', '리스닝', '리얼', '마지막',
    '머리', '메이커', '명작', '모타운', '뮤지션', '미러볼', '미스터', '보이스', '보컬', '보컬리스트',
    '분업', '사운드트랙', '소리', '소환', '수록곡', '순위', '스코어', '스탠다드', '스탠더드', '시간',
    '시업', '아까운', '언제', '올해', '월드', '월드뮤직', '유행', '이브', '일리네어', '입문', '작곡가',
    '작사', '제이', '좋아', '좋아하는', '좋은', '좋음', '주제가', '주제곡', '주차', '초이스', '최애',
    '코노', '코인', '콜라보', '코로나', '크루', '타이틀', '탐구생활', '트랙', '트로피', '파이', '파이팅',
    '팝스타', '프로듀서', '프롬', '플레이', '피쉬', '피아니스트', '피처링', '피쳐링', '필수', '필요할',
    '하이', '한잔', '함께', '해소', '혼성'
]


corrections = {
    '가벼운': '가볍게',
    '감미': '감미로운',
    '결혼식': '결혼',
    '경쾌한': '경쾌',   
    '고요': '고요한',
    '광고음악': '광고',
    '국힙': '힙합',
    '그루비': '그루브',
    '그리운':'그리움',
    '기타리스트': '기타',
    '꿀맛': '꿀',
    '꿀잠': '잠',
    '나른한': '나른함',
    '남성': '남자',
    '남친': '남자친구',
    '내적댄스': '댄스',
    '노랫말': '가사',
    '달콤한': '달콤',
    '댄스댄스': '댄스',
    '댄스팝': '댄스',
    '더운': '더위',
    '두둠칫': '둠칫둠칫',
    '드라이빙': '드라이브',
    '따뜻하게': '따뜻함',
    '따뜻한': '따뜻함',
    '따스한': '따뜻함',
    '따스함': '따뜻함',
    '라틴팝': '라틴',
    '래퍼': '랩',
    '러브': '사랑',
    '로맨스': '로맨틱',
    '리듬': '리드미컬',
    '맑게': '맑음',
    '맑은': '맑음',
    '메리크리스마스': '크리스마스',
    '멜랑꼴리': '멜랑콜리',
    '모닝': '아침',
    '모닝콜': '아침',
    '모던락': '모던',
    '모던록': '모던',
    '몸매': '다이어트',
    '미드': '미국',
    '발랄한': '발랄',
    '밤하늘': '밤',
    '방안': '방',
    '방콕': '방',
    '봄날': '봄',
    '봄맞이': '봄',
    '봄바람': '봄',
    '불면증': '불면',
    '브금': '배경음악',
    '빈티': '빈티지',
    '사랑스러운': '사랑',
    '산뜻한': '산뜻',
    '산소': '유산소',
    '상쾌한': '상쾌',
    '설레는': '설렘',
    '설레임': '설렘',
    '성탄절': '크리스마스',
    '세련된': '세련',
    '섹시한': '섹시',
    '센치한': '센치',
    '센치함': '센치',
    '센티': '센치',
    '수면': '숙면',
    '스윗': '달콤',
    '스윙재즈': '스윙',
    '스타일리시': '스타일',
    '슬픈': '슬픔',
    '슬플': '슬픔',
    '시원한': '시원',
    '신나게': '신나는',
    '신나': '신나는',
    '신나고': '신나는',
    '신난': '신나는',
    '신난다': '신나는',
    '신남': '신나는',
    '신비': '신비로운',
    '심심할': '심심',
    '쌀쌀': '쌀쌀한',
    '쓸쓸한': '쓸쓸함',
    '쓸쓸할': '쓸쓸함',
    '쓸쓸': '쓸쓸함',
    '아련한': '아련',
    '아픈': '아픔',
    '알엔비': '알앤비',
    '애니': '애니메이션',
    '어린이집': '어린이',
    '여름향기': '여름',
    '여자친구': '여친',
    '애인': '연인',
    '연주자': '연주',
    '영화음악': '영화',
    '옛사랑': '사랑',
    '외로': '외로움',
    '외로운': '외로움',
    '우울한': '우울',
    '우울할': '우울',
    '우울함': '우울',
    '일렉트로': '일렉',
    '일렉트로니카': '일렉',
    '일렉트로닉': '일렉',
    '일렉트로닉팝': '일렉',
    '일렉트로팝': '일렉',
    '잔잔': '잔잔한',
    '재즈팝': '재즈',
    '전자음악': '일렉',
    '조용': '조용한',
    '조용히': '조용한',
    '즐거움': '즐거운',
    '집콕': '집',
    '차안': '차',
    '청량감': '청량한',
    '청량함': '청량한',
    '초여름': '여름',
    '출근길': '출근',
    '출퇴근길': '출퇴근',
    '캐럴': '캐롤',
    '커피숍': '커피',
    '커피한잔': '커피',
    '모닝커피': '커피',
    '퇴근길': '퇴근',
    '트랜디': '트렌디',
    '트롯': '트로트',
    '파워워킹': '조깅',
    '팝락': '락',
    '팝랩': '랩',
    '팝음악': '팝',
    '펑키': '펑크',
    '편안함': '편안한',
    '편집': '편집샵',
    '평화로운': '평화',
    '포근한': '포근함',
    '퓨쳐': '퓨처',
    '하드락': '하드록',
    '하이네켄': '맥주',
    '하트': '사랑',
    '학기': '학교',
    '행복한': '행복',
    '헬스장': '헬스',
    '파이팅': '화이팅',
    '활기': '활기찬',
    '흑인음악': '흑인',
    '흥겨운': '흥',
    '흥얼흥얼': '흥',
    '힘들': '힘든',
    '힙': '힙합',
    '힙스터': '힙합',
    '힘': '힘나는'
}